*Author:wxz*  
*2018/08/06*

# Work
目前正在做一个把`Pytorch`框架的推荐算法改成`Mxnet`框架。  
`mxnet`框架发展得很快速，目前支持的编程语言有Python,Scala,R, Julia, Perl, Cpp。支持GPU编程，也支持多GPU编程。 支持Linux和MacOS，但是WindowOS还不支持。  
改编的过程还算顺利，因为两种框架的层的名字、定义基本相同，还有数据集类的定义也相同，基本不用改动，但是同时也碰到了很多坑，这些坑是这两个框架之间很不一样的地方，其中有一个初始化的坑卡了差不多2天。废话不多说，摆坑。

# 1.初始化方式不一样
pytorch的初始化可以在网络定义的时候直接初始化，但是mxnet的初始化必须在建立对象之后，由对象执行initialize()函数才能够初始化。下面给出代码逐步分析，为了让代码简洁，特意删去网络的前传过程，仅仅给出初始化过程：`__init__()`
## Pytorch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pdb

class NeuMF(nn.Module):
    def __init__(self, nb_users, nb_items,
                 mf_dim, mf_reg,
                 mlp_layer_sizes, mlp_layer_regs):
        # TODO: regularization?
        self.mf_user_embed = nn.Embedding(nb_users, mf_dim)
        self.mf_item_embed = nn.Embedding(nb_items, mf_dim)
        self.mlp_user_embed = nn.Embedding(nb_users, mlp_layer_sizes[0] // 2)
        self.mlp_item_embed = nn.Embedding(nb_items, mlp_layer_sizes[0] // 2)

        # pytorch在函数定义时就可以初始化函数，也就是说这个NeuMF类生成一个
        # 对象时就可以直接拥有初始化的权重。
        self.mf_user_embed.weight.data.normal_(0., 0.01)
        self.mf_item_embed.weight.data.normal_(0., 0.01)
        self.mlp_user_embed.weight.data.normal_(0., 0.01)
        self.mlp_item_embed.weight.data.normal_(0., 0.01)

net = NeuMF(nb_users, nb_items,
            mf_dim, mf_reg,
            mlp_layer_sizes, mlp_layer_regs) # net现在已经初始化了，可以直接输入数据

## Mxnet

In [ ]:
import numpy as np
from mxnet.gluon import nn
from mxnet import nd
import mxnet as mx

class NeuMF(nn.HybridBlock): #if using nn.Hybridblock, it will generate static graph
    def __init__(self, nb_users, nb_items,
                 mf_dim, mf_reg,
                 mlp_layer_sizes,
                 mlp_layer_regs,# mlp_layer_regs is a reconfirm
                 ctx):  # Indicate the context is CPU or GPU
        super(NeuMF, self).__init__()
        with self.name_scope():
            nb_mlp_layers = len(mlp_layer_sizes)  
            # 这里指定了四个层的初始化方式：mx.init.Normal()
            # 但是只是指定了方式而已，还未初始化
            self.mf_user_embed = nn.Embedding(nb_users, mf_dim,
                                          weight_initializer=mx.init.Normal())
            self.mf_item_embed = nn.Embedding(nb_items, mf_dim,
                                          weight_initializer=mx.init.Normal())
            self.mlp_user_embed = nn.Embedding(nb_users, mlp_layer_sizes[0] // 2,
                                           weight_initializer=mx.init.Normal())
            self.mlp_item_embed = nn.Embedding(nb_items, mlp_layer_sizes[0] // 2,
                                           weight_initializer=mx.init.Normal())
net = NeuMF(nb_users, nb_items,
            mf_dim, mf_reg,
            mlp_layer_sizes, mlp_layer_regs) # net现在还未初始化
net.initialize() # 使用了initialize()函数之后，net才算是真正初始化

# 2.自动求导的方式不一样
## Pytorch

In [ ]:
import torch
# pytorch里是将变量变成torch.autograd.Variable类型来自动求导的
user = torch.autograd.Variable(user, requires_grad=False)
item = torch.autograd.Variable(item, requires_grad=False)
label = torch.autograd.Variable(label, requires_grad=False)

outputs = model(user, item)
loss = criterion(outputs, label)
loss.backward()

## Mxnet

In [ ]:
import mxnet

ctx = mxnet.cpu()
user = nd.array(user,ctx=ctx)
item = nd.array(item,ctx=ctx)
label = nd.array(label,ctx=ctx)

# compute the gradient automatically
# mxnet里是将变量放到with autograd.record():
# 之下使其获得自动求导的能力
with autograd.record():
    outputs = model(user, item)
    loss = mxnet_criterion(outputs, label.T)

loss.backward()
trainer.step(bs)

# 3.使用GPU编程的定义操作不一样